In [1]:
import pandas as pd      
import numpy as np 
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings('ignore')
import xgboost as xgb

In [2]:
#pip install scikit-learn
#import sys
#!{sys.executable} -m pip install xgboost

In [3]:
def eval_metrics(actual, pred):
    rmse = np.sqrt(mean_squared_error(actual, pred))
    mae = mean_absolute_error(actual, pred)
    mse = mean_squared_error(actual, pred)
    score = r2_score(actual, pred)
    return print(" r2_score:", score, "\n","mae:", mae, "\n","mse:",mse, "\n","rmse:",rmse)

In [4]:
df = pd.read_csv("final_scout_not_dummy.csv")
df.head()

,make_model,body_type,price,vat,km,Type,Fuel,Gears,Comfort_Convenience,Entertainment_Media,...,Previous_Owners,hp_kW,Inspection_new,Paint_Type,Upholstery_type,Gearing_Type,Displacement_cc,Weight_kg,Drive_chain,cons_comb
0,Audi A1,Sedans,15770,VAT deductible,56013.0,Used,Diesel,7.0,"Air conditioning,Armrest,Automatic climate con...","Bluetooth,Hands-free equipment,On-board comput...",...,2.0,66.0,1,Metallic,Cloth,Automatic,1422.0,1220.0,front,3.8
1,Audi A1,Sedans,14500,Price negotiable,80000.0,Used,Benzine,7.0,"Air conditioning,Automatic climate control,Hil...","Bluetooth,Hands-free equipment,On-board comput...",...,1.0,141.0,0,Metallic,Cloth,Automatic,1798.0,1255.0,front,5.6
2,Audi A1,Sedans,14640,VAT deductible,83450.0,Used,Diesel,7.0,"Air conditioning,Cruise control,Electrical sid...","MP3,On-board computer",...,1.0,85.0,0,Metallic,Cloth,Automatic,1598.0,1135.0,front,3.8
3,Audi A1,Sedans,14500,VAT deductible,73000.0,Used,Diesel,6.0,"Air suspension,Armrest,Auxiliary heating,Elect...","Bluetooth,CD player,Hands-free equipment,MP3,O...",...,1.0,66.0,0,Metallic,Cloth,Automatic,1422.0,1195.0,front,3.8
4,Audi A1,Sedans,16790,VAT deductible,16200.0,Used,Diesel,7.0,"Air conditioning,Armrest,Automatic climate con...","Bluetooth,CD player,Hands-free equipment,MP3,O...",...,1.0,66.0,1,Metallic,Cloth,Automatic,1422.0,1135.0,front,4.1


In [5]:
web_list=["age","make_model","km","hp_kW"]

In [6]:
X=df[web_list]
y=df['price']
X=pd.get_dummies(X)

In [7]:
X.sample(5)

,age,km,hp_kW,make_model_Audi A1,make_model_Audi A2,make_model_Audi A3,make_model_Opel Astra,make_model_Opel Corsa,make_model_Opel Insignia,make_model_Renault Clio,make_model_Renault Duster,make_model_Renault Espace
7092,1.0,15756.0,110.0,0,0,0,1,0,0,0,0,0
10634,3.0,70968.0,125.0,0,0,0,0,0,1,0,0,0
428,3.0,69900.0,66.0,1,0,0,0,0,0,0,0,0
7179,1.0,28184.0,100.0,0,0,0,1,0,0,0,0,0
15752,1.0,4300.0,118.0,0,0,0,0,0,0,0,0,1


In [8]:
X.age.value_counts()

1.0    4526
0.0    4433
3.0    3679
2.0    3277
Name: age, dtype: int64

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

In [10]:
xgb_model = xgb.XGBRegressor()
xgb_model.fit(X_train, y_train)

XGBRegressor(base_score=0.5, booster='gbtree', callbacks=None,
             colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
             early_stopping_rounds=None, enable_categorical=False,
             eval_metric=None, gamma=0, gpu_id=-1, grow_policy='depthwise',
             importance_type=None, interaction_constraints='',
             learning_rate=0.300000012, max_bin=256, max_cat_to_onehot=4,
             max_delta_step=0, max_depth=6, max_leaves=0, min_child_weight=1,
             missing=nan, monotone_constraints='()', n_estimators=100, n_jobs=0,
             num_parallel_tree=1, predictor='auto', random_state=0, reg_alpha=0,
             reg_lambda=1, ...)

In [11]:
y_pred = xgb_model.predict(X_test)
eval_metrics(y_test, y_pred)

 r2_score: 0.9000448405208343 
 mae: 1489.8293307463478 
 mse: 5205724.942055583 
 rmse: 2281.605781474


In [12]:
from sklearn.ensemble import RandomForestRegressor
rf_model=RandomForestRegressor()
rf_model.fit(X_train, y_train)

RandomForestRegressor()

In [13]:
y_pred2 = rf_model.predict(X_test)

In [14]:
eval_metrics(y_test,y_pred2)

 r2_score: 0.8876034769284674 
 mae: 1515.0934698518577 
 mse: 5853678.65553514 
 rmse: 2419.4376734140396


## Deployment

In [15]:
import pickle
pickle.dump(xgb_model, open('xgb_modelc', 'wb'))
pickle.dump(rf_model, open('rf_modelc', 'wb'))

In [16]:
project_model= pickle.load(open('xgb_modelc', 'rb'))

In [17]:
columns=list(X.columns)
columns

['age',
 'km',
 'hp_kW',
 'make_model_Audi A1',
 'make_model_Audi A2',
 'make_model_Audi A3',
 'make_model_Opel Astra',
 'make_model_Opel Corsa',
 'make_model_Opel Insignia',
 'make_model_Renault Clio',
 'make_model_Renault Duster',
 'make_model_Renault Espace']

In [18]:
my_dict = {
    "age": 3,
    "hp": 105,
    "km": 10000,
    "car_model": 'A1'
}

In [19]:
df = pd.DataFrame.from_dict([my_dict])

In [20]:
df.head()

,age,hp,km,car_model
0,3,105,10000,A1


In [21]:
df = pd.get_dummies(df)
df

,age,hp,km,car_model_A1
0,3,105,10000,1


In [22]:
df = pd.get_dummies(df).reindex(columns=columns, fill_value=0)

In [23]:
df.head()

,age,km,hp_kW,make_model_Audi A1,make_model_Audi A2,make_model_Audi A3,make_model_Opel Astra,make_model_Opel Corsa,make_model_Opel Insignia,make_model_Renault Clio,make_model_Renault Duster,make_model_Renault Espace
0,3,10000,0,0,0,0,0,0,0,0,0,0


In [24]:
prediction = project_model.predict(df)
print(prediction)

[7516.3574]


In [25]:
print("The estimated price of your car is €{}. ".format(int(prediction[0])))

The estimated price of your car is €7516. 
